In [1]:
from kiteconnect.exceptions import *

from pyportmanlib import  *
auth_df = pd.read_excel('./auth_info.xlsx')
stocks_df = pd.read_excel('./stocks.xlsx')
global_exchange_type = 'NSE'
display(qgrid.show_grid(stocks_df))
# display(qgrid.show_grid(auth_df))

this_clients_group = my_clients_group(auth_df)

this_clients_group.do_login_df()

def make_global(x):
    global xy
    xy = x

def get_holding_qty(user_name,symbol_name):
    this_hld_list = globals()[user_name].get_holdings_list()
    this_qty = 0
    if(len(this_hld_list)>0):
        for one_hld in this_hld_list:
            if(symbol_name==one_hld.tradingsymbol):
                this_qty+=one_hld.quantity + one_hld.t1_quantity
    return this_qty

def get_position_qty(user_name,symbol_name):
    this_pos_list = globals()[user_name].get_positions_list()
    this_qty = 0
    if(len(this_pos_list)>0):
        for one_pos in this_pos_list:
            if(symbol_name==one_pos.tradingsymbol):
                this_qty+=one_pos.buy_quantity - one_pos.sell_quantity
    return this_qty

for one_user in this_clients_group.user_list:
    globals()[one_user.ac_name] = one_user


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [2]:
import PySimpleGUI as sg   
import pandas as pd
from collections import OrderedDict
sg.theme('LightBrown10')    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
####################################### GTT ##########################################
gtts_df = pd.read_excel('./plot_df/gtts.xlsx')
gtts_df = gtts_df[['user','gtt_gtt_status','gtt_gtt_type',
         'gtt_gtt_condition_tradingsymbol','gtt_gtt_condition_trigger_values_0',
         'gtt_gtt_condition_trigger_values_1','gtt_gtt_orders_0_quantity',
         'gtt_gtt_orders_0_transaction_type','gtt_gtt_id']]

gtts_df.rename(columns={'gtt_gtt_status':'status',
                        'gtt_gtt_type':'type',
                        'gtt_gtt_condition_tradingsymbol':'symbol',
                        'gtt_gtt_condition_trigger_values_0':'trigger_0',
                        'gtt_gtt_condition_trigger_values_1':'trigger_1',
                        'gtt_gtt_orders_0_quantity':'qty',
                        'gtt_gtt_orders_0_transaction_type':'transaction',
                        'gtt_gtt_id':'id',
                        },
          inplace=True, errors='raise')

gtt_user_list = gtts_df.user.to_list()
gtt_symbol_list = gtts_df.symbol.to_list()
gtt_id_list = gtts_df.id.to_list()
gtt_button_id_list = []
gtt_counter = 0
for one_symbol in gtt_symbol_list:
    this_gtt_button_id = str('Delete:'+ str(gtt_user_list[gtt_counter])+':'+gtt_symbol_list[gtt_counter]+':'+str(gtt_id_list[gtt_counter])) 
    gtt_button_id_list.append(this_gtt_button_id)
    gtt_counter+=1
    
gtts_main_list = []
gtts_header_list = list(gtts_df.columns.values)
gtts_header_widget_list = []
for one_header in gtts_header_list:
    gtts_header_widget_list.append(sg.Text(one_header, size=(10,1)))
gtts_header_widget_list.append(sg.Button('Cancel_All_GTTs'))
gtts_main_list.append(gtts_header_widget_list)
temp_counter = 0
for __,one_row in gtts_df.iterrows():
    sec_list = []
    for one_key in one_row.keys():
        sec_list.append(sg.Text(one_row[one_key], size=(10,1)))
    sec_list.append(sg.Button(gtt_button_id_list[temp_counter]))
    gtts_main_list.append(sec_list)
    temp_counter += 1
    
#### USER WISE gtts ###  
gtts_users_list_non_repeating = list(OrderedDict.fromkeys(gtts_df.user.to_list()))
gtts_users_wise_buttons_id_list = [ 'Delete:gtts:user:'+y for y in gtts_users_list_non_repeating ]
gtts_users_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in gtts_users_wise_buttons_id_list ]

gtts_main_list.append([sg.HSeparator()])
tp_counter = 0
total_counter = 0
temp_button_row = []
for one_button in gtts_users_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(gtts_users_wise_buttons_list)-1)):
        tp_counter = 0
        gtts_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1

#### SYMBOL WISE gtts ###
gtts_symbol_list_non_repeating = list(OrderedDict.fromkeys(gtts_df.symbol.to_list()))
gtts_symbol_wise_buttons_id_list = [ 'Delete:gtts:symbol:'+y for y in gtts_symbol_list_non_repeating ]
gtts_symbol_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in gtts_symbol_wise_buttons_id_list ]

total_counter = 0
gtts_main_list.append([sg.HSeparator()])
tp_counter = 0
temp_button_row = []
for one_button in gtts_symbol_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(gtts_symbol_wise_buttons_list)-1)):
        tp_counter = 0
        gtts_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1
    
############################3####### HOLDING #########################################
holding_df = pd.read_excel('./plot_df/holding.xlsx')

user_list = holding_df.user.to_list()
symbol_list = holding_df.hld_holding_tradingsymbol.to_list()
holding_button_id_list = []
holding_counter = 0
for one_symbol in symbol_list:
    this_hold_button_id = str('Exit:'+ str(user_list[holding_counter])+':hld:'+symbol_list[holding_counter]) 
    holding_button_id_list.append(this_hold_button_id)
    holding_counter+=1

holding_df = holding_df[['user','hld_tradingsymbol','hld_holding_opening_quantity',
                         'hld_holding_pnl','hld_day_change_percentage']]

holding_df.rename(columns={'hld_tradingsymbol':'symbol',
                            'hld_holding_opening_quantity':'qty',
                            'hld_holding_pnl':'pnl',
                            'hld_day_change_percentage':'ch_percent'
                             },
          inplace=True, errors='raise')

holding_df
holding_main_list = []
holding_header_list = list(holding_df.columns.values)
holding_header_widget_list = []
for one_header in holding_header_list:
    holding_header_widget_list.append(sg.Text(one_header, size=(10,1)))
holding_header_widget_list.append(sg.Button('EXIT_ALL_HOLDINGS'))
holding_main_list.append(holding_header_widget_list)
temp_counter = 0
for __,one_row in holding_df.iterrows():
    sec_list = []
    for one_key in one_row.keys():
        sec_list.append(sg.Text(one_row[one_key], size=(10,1)))
    sec_list.append(sg.Button(holding_button_id_list[temp_counter]))
    holding_main_list.append(sec_list)
    temp_counter+=1

#### USER WISE HOLDING ###  
hld_users_list_non_repeating = list(OrderedDict.fromkeys(holding_df.user.to_list()))
hld_users_wise_buttons_id_list = [ 'Exit:hld:user:'+y for y in hld_users_list_non_repeating ]
hld_users_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in hld_users_wise_buttons_id_list ]

holding_main_list.append([sg.HSeparator()])
tp_counter = 0
total_counter = 0
temp_button_row = []
for one_button in hld_users_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(hld_users_wise_buttons_list)-1)):
        tp_counter = 0
        holding_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1

#### SYMBOL WISE HOLDING ###
hld_symbol_list_non_repeating = list(OrderedDict.fromkeys(holding_df.symbol.to_list()))
hld_symbol_wise_buttons_id_list = [ 'Exit:hld:symbol:'+y for y in hld_symbol_list_non_repeating ]
hld_symbol_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in hld_symbol_wise_buttons_id_list ]

total_counter = 0
holding_main_list.append([sg.HSeparator()])
tp_counter = 0
temp_button_row = []
for one_button in hld_symbol_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(hld_symbol_wise_buttons_list)-1)):
        tp_counter = 0
        holding_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1

################################## POSITION #########################################
position_df = pd.read_excel('./plot_df/positions.xlsx')
position_df['qty'] = position_df['pos_buy_quantity'] - position_df['pos_sell_quantity']
position_df.rename(columns={'pos_position_average_price': 'avg_price',
                   'pos_tradingsymbol': 'symbol',
                   'pos_position_value': 'value',
                   'pos_position_pnl': 'pnl',
                   'pos_unrealised': 'unreal',},
          inplace=True, errors='raise')

position_df = position_df[['user','symbol','qty',
                           'avg_price','value',
                           'pnl','unreal']]

user_list = position_df.user.to_list()
symbol_list = position_df.symbol.to_list()
position_button_id_list = []
position_counter = 0
for one_symbol in symbol_list:
    this_hold_button_id = str('Exit:'+ str(user_list[position_counter])+':'+'pos'+':'+symbol_list[position_counter]) 
    position_button_id_list.append(this_hold_button_id)
    position_counter+=1
position_button_id_list

position_main_list = []
position_header_list = list(position_df.columns.values)
position_header_widget_list = []

for one_header in position_header_list:
    position_header_widget_list.append(sg.Text(one_header, size=(10,1)))
position_main_list.append(position_header_widget_list)
position_header_widget_list.append(sg.Button('EXIT_ALL_POSITIONS'))
temp_counter = 0
    
for __,one_row in position_df.iterrows():
    sec_list = []
    for one_key in one_row.keys():
        sec_list.append(sg.Text(one_row[one_key], size=(10,1)))
    sec_list.append(sg.Button(position_button_id_list[temp_counter]))
    position_main_list.append(sec_list)
    temp_counter+=1
    

## USER WISE POSITION BUTTON ##
pos_users_list_non_repeating = list(OrderedDict.fromkeys(position_df.user.to_list()))
pos_users_wise_buttons_id_list = [ 'Exit:pos:user:'+y for y in pos_users_list_non_repeating ]
pos_users_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in pos_users_wise_buttons_id_list ]

position_main_list.append([sg.HSeparator()])
tp_counter = 0
total_counter = 0
temp_button_row = []
for one_button in pos_users_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(pos_users_wise_buttons_list)-1)):
        tp_counter = 0
        position_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1
### SYMBOL WISE POSITION  ### 
pos_symbol_list_non_repeating = list(OrderedDict.fromkeys(position_df.symbol.to_list()))
pos_symbol_wise_buttons_id_list = ['Exit:pos:symbol:'+y for y in pos_symbol_list_non_repeating ]
pos_symbol_wise_buttons_list  = [ sg.Button(y, size=(25,1)) for y in pos_symbol_wise_buttons_id_list ]

position_main_list.append([sg.HSeparator()])
tp_counter = 0
total_counter = 0
temp_button_row = []
for one_button in pos_symbol_wise_buttons_list:
    temp_button_row.append(one_button)
    if(tp_counter%5==0 or total_counter==(len(pos_symbol_wise_buttons_list)-1)):
        tp_counter = 0
        position_main_list.append(temp_button_row)
        temp_button_row = []
    tp_counter+=1
    total_counter+=1
    
####################################### STOCKS ##################################### 
stock_df = pd.read_excel('./stocks.xlsx')

stock_main_list = []
stock_header_list = list(stock_df.columns.values)
stock_header_widget_list = []
for one_header in stock_header_list:
    stock_header_widget_list.append(sg.Text(one_header, size=(10,1)))
stock_main_list.append(stock_header_widget_list)

user_name_list_for_stock = []

for one_user in this_clients_group.user_list:
    user_name_list_for_stock.append(one_user.ac_name)

for __,one_row in stock_df.iterrows():
    sec_list = []
    for one_key in one_row.keys():
        sec_list.append(sg.Text(one_row[one_key], size=(10,1)))
    
    for each_user in user_name_list_for_stock:
        sec_list.append(sg.Button(each_user,key=each_user+':'+one_row['symbol']+':buy_new:'+str(one_row['percent'])))
    
    stock_main_list.append(sec_list)
    
####@@@@@@@@@@@@@@@@@@@@@@@@@@ MAIN PROCESS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
layout4 = [[sg.Button('Save Experience Details')]]
tabgrp = [[sg.TabGroup([[sg.Tab('GTT', gtts_main_list,),
                    sg.Tab('Holding', holding_main_list),
                    sg.Tab('Position', position_main_list),
                    sg.Tab('Stocks', stock_main_list),]])]]  

window =sg.Window("Tabs",
                   resizable=True,
                   finalize=True).Layout([[sg.Column(tabgrp, 
                                                     size=(1366,720),
                                                     scrollable=True)]])


####@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


######################## EVENT FUNCTION MANAGEMENT ################################
while True:
    event, values = window.read()
    event_split_list=[]
    if(event!=None):
        event_split_list = event.split(':')
######################################## GTT #####################################    
    #### DELETING SINGLE GTT  ####
    for one_gtt_id in gtt_button_id_list:
        if event ==one_gtt_id :
            print(f'Delete single gtt id : {event_split_list[3]}')
            
    ### CANCEL ALL GTTs ###
    if event == 'Cancel_All_GTTs':
        print('Cancel_All_GTTs')
        
    ## DELETE GTTs FOR A USER ####
    if (('Delete' in event_split_list) and ('user' in event_split_list)):
        print(f'Delete GTTs for user {event_split_list[3]}')
        
    ## DELETE GTTs FOR A SYMBOL ####
    if (('Delete' in event_split_list) and ('symbol' in event_split_list)):
        print(f'Delete GTTs for symbol {event_split_list[3]}')
        
######################################## HOLDING ####################################      
    #### EXITING SINGLE HOLDING ####
    for one_hld in holding_button_id_list:
        if event == one_hld:
            userip = event_split_list[1]
            symbol = event_split_list[3]
            this_hld_qty = get_holding_qty(userip,symbol)
            print(f'Exit single holding : {userip} : {symbol} : {this_hld_qty}')
            try:
                this_message = globals()[userip].zerodha_user.place_order(globals()[userip].zerodha_user.VARIETY_REGULAR,
                                                                            global_exchange_type,symbol,
                                                                            globals()[userip].zerodha_user.TRANSACTION_TYPE_SELL,
                                                                            this_hld_qty,
                                                                            globals()[userip].zerodha_user.PRODUCT_CNC,
                                                                            globals()[userip].zerodha_user.ORDER_TYPE_MARKET
                                                                           )
                print(this_message)
            except InputException as ipxcp:
                print(ipxcp)
            except GeneralException as ipxcp:
                print(ipxcp)
    
    ### EXIT ALL HOLDINGS@MARKET ####
    if event == 'EXIT_ALL_HOLDINGS':
        print('EXIT_ALL_HOLDINGS')
        
    ## DELETE HOLDINGS FOR A USER ####
    if (('Exit' in event_split_list) and ('user' in event_split_list) and ('hld' in event_split_list)):
        print(f'Exit holdings for user {event_split_list[3]}')
        
    ## DELETE HOLDINGS FOR A SYMBOL ####
    if (('Exit' in event_split_list) and ('symbol' in event_split_list) and ('hld' in event_split_list)):
        print(f'Exit holdings for symbol {event_split_list[3]}')
    
######################################## POSITION ####################################      
    #### EXITING SINGLE POSITIONS ####
    for one_pos in position_button_id_list:
        if event == one_pos:
            userip = event_split_list[1]
            symbol = event_split_list[3]
            this_pos_qty = get_position_qty(userip,symbol)
            print(f'Exit single position : {userip} : {symbol} : {this_pos_qty}')
            try:
                
                this_message = globals()[userip].zerodha_user.place_order( globals()[userip].zerodha_user.VARIETY_REGULAR,
                                                                            global_exchange_type,symbol,
                                                                            globals()[userip].zerodha_user.TRANSACTION_TYPE_SELL,
                                                                            this_pos_qty,
                                                                            globals()[userip].zerodha_user.PRODUCT_CNC,
                                                                            globals()[userip].zerodha_user.ORDER_TYPE_MARKET
                                                                           )
                print(this_message)
            except InputException as ipxcp:
                print(ipxcp)
            except GeneralException as ipxcp:
                print(ipxcp)
            
            
    
    ### EXIT ALL POSITIONS@MARKET ####
    if event == 'EXIT_ALL_POSITIONS':
        print('EXIT_ALL_POSITIONS')
        
    ## DELETE POSITIONS FOR A USER ####
    if (('Exit' in event_split_list) and ('user' in event_split_list) and ('pos' in event_split_list)):
        print(f'Exit position for user {event_split_list[3]}')
        
    ## DELETE POSITIONS FOR A SYMBOL ####
    if (('Exit' in event_split_list) and ('symbol' in event_split_list) and ('pos' in event_split_list)):
        print(f'Exit position for symbol {event_split_list[3]}')
    
    if event == sg.WIN_CLOSED:
        print("I am done")
        break
        
    if event == 'add_button':
        this_column_ip.AddRow(sg.InputText(),)
        this_column_ip.Update([sg.InputText()])
        
        
################################# STOCKS PAGE ######################################
    if('buy_new' in event_split_list):
        
        userip = event_split_list[0]
        symbol = event_split_list[1]
        percent = float(event_split_list[3])
        
        avail_margin = globals()[userip].zerodha_user.margins()['equity']['available']['cash']
        this_ltp = globals()[userip].zerodha_user.quote(f'NSE:{symbol}')[f'NSE:{symbol}']['last_price']
        this_qty = int((avail_margin*(percent/100))/this_ltp)
        
        print(f'Exit single position : {userip} : {symbol} : {this_qty}')
        try:
            this_message = globals()[userip].zerodha_user.place_order( globals()[userip].zerodha_user.VARIETY_REGULAR,
                                                                        global_exchange_type,symbol,
                                                                        globals()[userip].zerodha_user.TRANSACTION_TYPE_BUY,
                                                                        this_qty,
                                                                        globals()[userip].zerodha_user.PRODUCT_CNC,
                                                                        globals()[userip].zerodha_user.ORDER_TYPE_MARKET
                                                                       )
            print(this_message)
        except InputException as ipxcp:
            print(ipxcp)
        except GeneralException as ipxcp:
            print(ipxcp)


window.Close()

In [ ]:
window.Close()

In [ ]:
import PySimpleGUI as sg
from alpha_vantage.timeseries import TimeSeries
import time

sg.theme('BluePurple')

layout = [[sg.Text('Last Price', size=(20, 2), justification='center')],
          [sg.Text('', size=(10, 2), font=('Helvetica', 20),
                   justification='center', key='quote')]]

window = sg.Window('MikeQuote', layout)
av = TimeSeries(key = 'key')
spy, _ = av.get_quote_endpoint(symbol='SPY')
last = spy['05. price']
yest = spy['08. previous close']
wait = 1  # Wait is in seconds

while True:
    event, values = window.read(timeout=10)
    if event in (None, 'Quit'):
        break
    spy, _ = av.get_quote_endpoint(symbol='SPY')
    last = spy['05. price']
    window['quote'].update(last)
    time.sleep(wait)


In [ ]:
###### TABLE ADD ROW EXAMPLE ####

import PySimpleGUI as sg
# import PySimpleGUIQt as sg                # runs on Qt too

def table_example():
    data = [['a1', 'b1', 'c1', 'd1'],
            ['a2', 'b2', 'c2', 'd2'],
            ['a3', 'b3', 'c3', 'd3'],
            ['a4', 'b4', 'c4', 'd4']]
    header_list = ['Col1', 'Col2', 'Col3', 'Col4']

    layout = [[sg.Table(values=data, headings=header_list, auto_size_columns=True,justification='center',
                        num_rows=min(len(data), 20), display_row_numbers=True, key='table')],
              [sg.Button('Add Row'), sg.CButton('Close')]]

    window = sg.Window('Simple Table', layout)
    while True:
        event, values = window.Read()
        if event in ('Close', None): break
        if event == 'Add Row':
            layout2 = [[sg.Text('Col1:', size=(8, 1)), sg.InputText(key='C1')],
                       [sg.Text('Col2:', size=(8, 1)), sg.InputText(key='C2')],
                       [sg.Text('Col3:', size=(8, 1)), sg.InputText(key='C3')],
                       [sg.Text('Col4:', size=(8, 1)), sg.InputText(key='C4')],
                       [sg.Button('Add'), sg.Button('Insert Pasted Values'), sg.Button('Close')]]

            window2 = sg.Window('Row Addition', layout2)
            while True:
                event2, v2 = window2.Read()
                print(event2, v2)
                if event2 in ('Close', None): break
                elif event2 == 'Add':
                    data.append([v2['C1'], v2['C2'], v2['C3'], v2['C4']])
                    window.FindElement('table').Update(values=data, num_rows=len(data))
                    break
                elif event2 == 'Insert Pasted Values':
                    for key, value in {'C1': 'a5', 'C2': 'b5', 'C3': 'c5', 'C4': 'd5'}.items():
                        window2.Element(key).Update(value)
            window2.Close()
table_example()

In [ ]:
window.AllKeysDict.keys()

In [ ]:
window2.Close()